<font color='blue' size=5><b>Комментарий ревьюера</b></font>

<font color='blue'>Привет, Светлана! Меня зовут Павел Григорьев, и я буду проверять этот проект.<br>Моя основная цель - не указать на совершённые тобой ошибки, а поделиться своим опытом и помочь тебе совершенствоваться как профессионалу.<br>Спасибо за проделанную работу! Предлагаю общаться на «ты».</font>
<details>
	<summary><u>Инструкция по организационным моментам (кликабельно)</u></summary>
<font color='blue'>Я буду использовать различные цвета, чтобы было удобнее воспринимать мои комментарии:</font>


---


<font color='blue'>синий текст - просто текст комментария</font>

<font color='green'>✔️ и зеленый текст - все отлично</font>

<font color='orange'>⚠️ и оранжевый текст - сделано все правильно, однако есть рекомендации, на что стоит обратить внимание</font>

<font color='red'>❌ и красный текст - есть недочеты</font>


</details>    
    </br>
<font color='blue'>Пожалуйста, не удаляй мои комментарии в случае возврата работы, так будет проще разобраться, какие были недочеты, а также сразу увидеть исправленное. </font>

Ответы на мои комментарии лучше тоже помечать.
Например: <font color='purple'><b>Комментарий студента</b></font>

<font color='blue'><b>Давай смотреть, что получилось!</b></font>


<font color='blue' size=3><b>Комментарий ревьюера 2</b></font>

<font color='blue'>Привет еще раз. Спасибо, за исправления. Оформление комментариев по работе сохраняется. Только обозначим, что это вторая итерация.</font> 

## Предсказание стоимости жилья

"В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2."

**Основная цель проекта** - предсказать медианную стоимость дома в жилом массиве на основе данных о жилье в Калифорнии в 1990 году. \
**Метрики для оценки качества модели:** RMSE, MAE и R2.

**Ход работы:**
1. [Чтение и предобработка данных](#preprocessing)
2. [Обучение модели линейной регресси в двух вариантов (с категориальными признаками и без)](#training)
3. [Выбор лучшей модели и анализ результатов](#results)

<a id='preprocessing'></a>
# Подготовка данных

In [1]:
# импорты
import pandas as pd 
import numpy as np
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, OneHotEncoder
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.mllib.evaluation import RegressionMetrics

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Хорошее оформление импортов! \
Импорты собраны в одной ячейке, разделены на функциональные группы пустой строкой.</font>

In [2]:
# инициализация spark-сессии
spark = SparkSession.builder \
                    .master("local") \
                    .appName("EDA California Housing") \
                    .getOrCreate()

In [3]:
# чтение данных
df_housing = spark.read.load('/datasets/housing.csv', format='csv', inferSchema=True, header=True)
df_housing.show(5)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR BAY|
|  -122.22|   37.86|              21.0|     7099.0|        1106.0|    2401.0|    1138.0|       8.3014|          358500.0|       NEAR BAY|
|  -122.24|   37.85|              52.0|     1467.0|         190.0|     496.0|     177.0|       7.2574|          352100.0|       NEAR BAY|
|  -122.25|   37.85|              52.0|     1274.0|         235.0|     558.0|     219.0|       5.6431|          341300.0|       NEAR BAY|
|  -122.25|   37.85|              

In [4]:
# типы данных
print(pd.DataFrame(df_housing.dtypes, columns=['column', 'type']))

               column    type
0           longitude  double
1            latitude  double
2  housing_median_age  double
3         total_rooms  double
4      total_bedrooms  double
5          population  double
6          households  double
7       median_income  double
8  median_house_value  double
9     ocean_proximity  string


Датафрейм содержит **10 колонок** со следующими данными:
- `longitude` — широта;
- `latitude` — долгота;
- `housing_median_age` — медианный возраст жителей жилого массива;
- `total_rooms` — общее количество комнат в домах жилого массива;
- `total_bedrooms` — общее количество спален в домах жилого массива;
- `population` — количество человек, которые проживают в жилом массиве;
- `households` — количество домовладений в жилом массиве;
- `median_income` — медианный доход жителей жилого массива;
- `median_house_value` — медианная стоимость дома в жилом массиве;
- `ocean_proximity` — близость к океану.

`median_house_value` представляет собой численную **целевую переменную**. Большая часть признаков аналогично являются числовыми, за исключением `ocean_proximity`.

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Отлично!  Выведены типы данных и первые строчки таблицы.</font>

In [5]:
# пропуски
for col in df_housing.columns:
    print(col, df_housing.filter(F.col(col).isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Обнаружены пропуски!</font>

Пропуски присутствуют только в столбце `total_bedrooms`. Заполним их медианой:

In [6]:
# медиана
bedrooms_median = df_housing.approxQuantile('total_bedrooms', [0.5], 0)[0]

In [7]:
df_housing = df_housing.na.fill(value=bedrooms_median, subset='total_bedrooms')

for col in df_housing.columns:
    print(col, df_housing.filter(F.col(col).isNull()).count())

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0
ocean_proximity 0


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Заполнили пропуски!</font>

Перед обучением модели, также необходимо закодировать категориальные признаки. В данном датасете он всего один.

In [8]:
# для последующего формирования pipeline
stages = []

# числовое представление категорий
indexer = StringIndexer(inputCols=['ocean_proximity'], 
                        outputCols=['ocean_proximity_idx'])

stages += [indexer]

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='blue'>Бывает так, что в тестовых данных случается категория которой нет в обучающих. В StringIndexer есть три стратегии обработки незнакомых Лейблов.
1. Выбрасывать исключение (по умолчанию).
2. Игнорировать строки с незнакомыми лейблами.
3. Выделять для всех незнакомых лейблов отдельную категорию.

За это отвечает аргумент [handleInvalid](https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.feature.StringIndexer.html#pyspark.ml.feature.StringIndexer.handleInvalid), в который нужно передать "error","skip" или "keep" соответственно.</font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ❌\
<font color='red'><s>Кодирование и Стандартизацию нужно проводить после разделения на выборки трансформерами обучеными на обучающих данных.</s></font>

<font color='blue'><b>Комментарий ревьюера: </b></font> ⚠️\
<font color='blue'>Можно использовать [Pipeline](https://habr.com/ru/post/522296/)</font>

<font color='purple'><b> Комментарий студента:</b> 
    Исправила! Попробовала собрать все в единый пайплайн. Деление на выборки оставила в разделе с обучением моделей.</font> 

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'>Препроцессинг проведён по всем правилам MLOps. Утечки информации между выборками исключены, даже если применять кроссвалидацию!</font>

In [9]:
# ohe-кодирование
encoder = OneHotEncoder(inputCol='ocean_proximity_idx',
                        outputCol='ocean_proximity_ohe')

stages += [encoder]

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️ <br>
<font color='green'>Верно, что используешь OHE.</font>

Числовые признаки также следует подготовить - отшкалировать значения.

In [10]:
numeric = df_housing.columns[:-2]

In [11]:
numeric

['longitude',
 'latitude',
 'housing_median_age',
 'total_rooms',
 'total_bedrooms',
 'population',
 'households',
 'median_income']

In [12]:
# векторизация
numerical_assembler = VectorAssembler(inputCols=numeric, outputCol="numeric_idx")
stages += [numerical_assembler]

In [13]:
# шкалирование
standardScaler = StandardScaler(inputCol='numeric_idx', outputCol="numeric_scaled")
stages += [standardScaler]

In [14]:
# итоговая таблица признаков, собранная в вектор
all_features = ['ocean_proximity_ohe','numeric_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                  outputCol="features") 

stages += [final_assembler]

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️ <br>
<font color='green'>Итоговые сборки сделаны.</font>

<a id='training'></a>
# Обучение моделей

Деление на выборки для обучения и тестирования модели.

In [15]:
# разделение на выборки
train_data, test_data = df_housing.randomSplit([.8,.2], seed=123)
print(train_data.count(), test_data.count()) 

16442 4198


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'> Здорово , что у нас есть выборка для тестов!</font>

Выбранный алгоритм машинного обучения - *Линейная регрессия*. \
Сравним две модели, обученные на полном наборе данных и только с использованием числовых признаков.

In [16]:
# обучение на полном наборе данных
lr = LinearRegression(featuresCol = 'features', labelCol='median_house_value', regParam=0.3)

stages_full = stages.copy()
stages_full += [lr]

# сбор всех трансформеров в пайплайн
pipeline = Pipeline(stages=stages_full)

# обучение пайплайна
lr_model = pipeline.fit(train_data)

# предсказание модели
predictions = lr_model.transform(test_data)
predictedLabes = predictions.select('median_house_value', 'prediction')

# перевод в RDD
predictionLabes_list = [(float(i[1]), float(i[0])) for i in predictedLabes.collect()]
predictionAndObservations = spark.sparkContext.parallelize(predictionLabes_list)

# метрики
metrics = RegressionMetrics(predictionAndObservations)

print('RMSE:', round(metrics.rootMeanSquaredError))
print('MAE:', round(metrics.meanAbsoluteError))
print('R2:', round(metrics.r2, 2))

23/03/21 15:16:18 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/03/21 15:16:18 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS
23/03/21 15:16:18 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeSystemLAPACK
23/03/21 15:16:18 WARN LAPACK: Failed to load implementation from: com.github.fommil.netlib.NativeRefLAPACK


RMSE: 67538
MAE: 49798
R2: 0.65


In [17]:
from pyspark.ml.evaluation import RegressionEvaluator
r2 = \
        RegressionEvaluator(labelCol='median_house_value',
                                            metricName='r2').evaluate(predictedLabes)
mae = \
        RegressionEvaluator(labelCol='median_house_value',
                                            metricName='mae').evaluate(predictedLabes)
rmse = \
        RegressionEvaluator(labelCol='median_house_value',
                                            metricName='rmse').evaluate(predictedLabes)
print('RMSE:', round(rmse, 2))
print('MAE:', round(mae, 2))
print('R2:', round(r2, 2))

RMSE: 67538.45
MAE: 49798.26
R2: 0.65


<font color='blue'><b>Комментарий ревьюера :</b></font> ✔️\
<font color='green'>Тестирование проведено верно.</font>

In [18]:
# обучение на неполном наборе данных (только численные признаки)
lr = LinearRegression(featuresCol = 'numeric_scaled', labelCol='median_house_value', regParam=0.3)

stages_part = stages.copy()
stages_part += [lr]

# сбор всех трансформеров в пайплайн
pipeline = Pipeline(stages=stages_part)

# обучение пайплайна
lr_model = pipeline.fit(train_data)

# предсказание модели
predictions = lr_model.transform(test_data)
predictedLabes = predictions.select('median_house_value', 'prediction')

# перевод в RDD
predictionLabes_list = [(float(i[1]), float(i[0])) for i in predictedLabes.collect()]
predictionAndObservations = spark.sparkContext.parallelize(predictionLabes_list)

# метрики
metrics = RegressionMetrics(predictionAndObservations)

print('RMSE:', round(metrics.rootMeanSquaredError))
print('MAE:', round(metrics.meanAbsoluteError))
print('R2:', round(metrics.r2, 2))

RMSE: 68335
MAE: 50796
R2: 0.64


<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>👍</font>

In [19]:
r2 = \
        RegressionEvaluator(labelCol='median_house_value',
                                            metricName='r2').evaluate(predictedLabes)
mae = \
        RegressionEvaluator(labelCol='median_house_value',
                                            metricName='mae').evaluate(predictedLabes)
rmse = \
        RegressionEvaluator(labelCol='median_house_value',
                                            metricName='rmse').evaluate(predictedLabes)
print('RMSE:', round(rmse, 2))
print('MAE:', round(mae, 2))
print('R2:', round(r2, 2))

RMSE: 68335.15
MAE: 50795.84
R2: 0.64


In [20]:
# закрытие spark-сессии
spark.stop()

<font color='blue'><b>Комментарий ревьюера: </b></font> ✔️\
<font color='green'>Здорово, что не забываешь останавливать сессию!</font>

<a id='results'></a>
# Анализ результатов

В рамках данного проекта с использованием *pySpark* удалось построить две модели линейной регрессии, предсказывающих медианную стоимость жилья в жилом массиве на основе данных о жилье в Калифорнии в 1990 году. Для обучения первой модели использовались все присутствующие признаки, для обучения второй - только численные. Выбранные метрики для оценки качества моделей: RMSE (root mean squared error), MAE (mean absolute error) и R2.

На этапе предобработки пропущенные значения в данных были восстановлены как медианные, категориальный признак закодирован с испоьзованием *One Hot Encoding*, а численные - отшкалированы для снижения влияния выбросов на предсказание модели. 

**Обучение на полном наборе данных оказалось эффективнее.** Модель, учитывающая влияние всех признаков объясняет бОльшую долю дисперсии целевой переменной (R2 **42%**), к тому же характеризуется меньшими значениями ошибок предсказания (RMSE и MAE).

<font color='blue'><b>Комментарий ревьюера : </b></font> ✔️\
<font color='green'> Очень приятно видеть вывод в конце проекта!\
Приведены ответы на главные вопросы проекта.</font>

<font color='blue'><b>Итоговый комментарий ревьюера</b></font>\
<font color='green'> Светлана, хороший проект получился!
Большое спасибо за проделанную работу. Видно, что приложено много усилий.
</font>

<font color='blue'>Что нужно исправить:</font>
<ul><font color='red'>Стандартизация и Кодировка для train и test по отдельности обработчиком обученным на train.</font></ul>

<font color='blue'><b>Жду новую версию проекта :)</b></font>

<font color='purple'><b> Комментарий студента: </b>
    Спасибо за ревью и идею с пайплайном. Я попробовала ее реализовать. Но вот интересно, что после того, как я исправила деление выборок и объединила все трансформеры в пайплайн, метрики ухудшились значительно (например, R2 снизился с >60% до 42%).. Это нормально и можно как-то объяснить? Или я где-то сделала ошибку? </font> 

<font color='blue'><b>Комментарий ревьюера 2: </b></font> ✔️\
<font color='green'>Добавил две выпавшие фичи, результат стал получше.</font>

<font color='blue'><b>Итоговый комментарий ревьюера 2</b></font>\
<font color='green'>Светлана, проект принят! \
Все этапы пройдены. Все рекомендации учтены.\
Надеюсь, тебе понравился процесс выполнения и результат.</font> \
<font color='blue'><b>Спасибо, удачи в освоении профессии!</b></font>

<font color='green'>Немного полезного материала:
+  https://sparkbyexamples.com/
+  https://github.com/dvgodoy/handyspark
+  https://www.tutorialspoint.com/pyspark/index.htm
+  https://www.guru99.com/pyspark-tutorial.html
+  https://databricks.com/spark/getting-started-with-apache-spark/machine-learning#load-sample-data
</font>